In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install fairseq
!pip install fastBPE

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.8 MB/s 
     |████████████████████████████████| 86 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.8 MB/s 
     |████████████████████████████████| 596 kB 61.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 4.8 MB/s 
     |████████████████████████████████| 92 kB 12.2 MB/s 
     |████████████████████████████████| 151 kB 76.5 MB/s 
     |████████████████████████████████| 117 kB 77.3 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=4e47e52057dada432372

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=483885 sha256=058b867b2b12185626e9c2d329a92e57fe404275e34fe10195e7500fe411b42c
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE


In [3]:
!git clone https://github.com/vncorenlp/VnCoreNLP
!pip install vncorenlp

Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 218 (delta 2), reused 1 (delta 0), pack-reused 212
Receiving objects: 100% (218/218), 214.22 MiB | 40.72 MiB/s, done.
Resolving deltas: 100% (78/78), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 3.9 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=69051377f6df78c69357131afe2a5c5bf4ec1ae04ed57e2eb1dcc5f5c08808bf
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp


In [4]:
from transformers import RobertaForSequenceClassification, RobertaConfig
from tensorflow.keras.preprocessing.sequence import pad_sequences
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from vncorenlp import VnCoreNLP
import pandas as pd
import argparse
import torch
import re

In [7]:
path_config  = '/content/drive/MyDrive/Shopee_comment_ranking/4/config.json'
path_model = '/content/drive/MyDrive/Shopee_comment_ranking/4/pytorch_model.bin'
path_bpe = '/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/bpe.codes'
path_vocab = '/content/drive/MyDrive/Shopee_comment_ranking/PhoBERT_base_transformers/dict.txt'

def get_model(path_model= None, path_config = None, path_bpe = None, path_vocab = None):
  config = RobertaConfig.from_pretrained(
      path_config, from_tf=False, num_labels = 2, output_hidden_states=False,
  )
  BERT_SA_NEW = RobertaForSequenceClassification.from_pretrained(
      path_model,
      config=config
  )
  BERT_SA_NEW.cuda()
  BERT_SA_NEW.eval()


  try:
    parser = argparse.ArgumentParser()
    parser.add_argument('--bpe-codes', 
        default=path_bpe,
        required=False,
        type=str,
        help='path to fastBPE BPE'
    )
    args, unknown = parser.parse_known_args()
    bpe = fastBPE(args)
  except:
    bpe = None
    print("load bpe fail")

  try:
    vocab = Dictionary()
    vocab.add_from_file(path_vocab)
  except:
    vocab=None
    print('load vocab fail')
  return BERT_SA_NEW, bpe, vocab

model, bpe, vocab = get_model(path_model, path_config, path_bpe, path_vocab)

In [8]:
rdrsegmenter = VnCoreNLP("VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [9]:
macp = pd.read_excel('/content/drive/MyDrive/Shopee_comment_ranking/teencode.xlsx')

teencod = {}

dung = macp['Từ đúng'].tolist()
code_char = macp['Teencod'].tolist()
for i in range(len(code_char)):
  teencod[code_char[i]] = dung[i]
print(teencod)


def del_test(text):
  year = ['năm 2021', 'năm 2020', 'năm 2019', 'năm 2018', 'năm 2017', 'năm 2016', 'năm 2015', 'năm 2014', 'năm 2013', 'năm 2012', 'năm 2011', 'năm 2010', 'Năm 2021', 'Năm 2020', 'Năm 2019', 'Năm 2018', 'Năm 2017', 'Năm 2016', 'Năm 2015', 'Năm 2014', 'Năm 2013', 'Năm 2012', 'Năm 2011', 'Năm 2010', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010']
  month = ['tháng 1', 'tháng 2', 'tháng 3', 'tháng 4', 'tháng 5', 'tháng 6', 'tháng 7', 'tháng 8', 'tháng 9', 'tháng 10', 'tháng 11', 'tháng 12', 'Tháng 1', 'Tháng 2', 'Tháng 3', 'Tháng 4', 'Tháng 5', 'Tháng 6', 'Tháng 7', 'Tháng 8', 'Tháng 9', 'Tháng 10', 'Tháng 11', 'tháng 12']
  quy = ['quý 1', 'quý 2', 'quý 3', 'quý 4', 'Quý 1', 'Quý 2', 'Quý 3', 'Quý 4']
  text = text.replace('Covid-19', 'Covid')
  word_segmented_text = rdrsegmenter.ner(text)[0]
  for char, typ in word_segmented_text:
    if typ == 'B-ORG' or typ == 'I-ORG' or typ == 'B-PER' or typ == 'I-PER':
      char = char.replace('_', ' ')
      text = text.replace(char, 'name')
    if typ == "B-LOC" or typ == "I-LOC":
      if char != 'VN':
        char = char.replace('_', ' ')
        text = text.replace(char,'loc')
    if typ == 'O':
      if len(re.findall('\d*\.?\,?\d+\%', char)) > 0:
        text = text.replace(char, 'percent')
      # if len(re.findall('\s?\(?[A-Z]{3,4}\)?\s?', char)) > 0 and char != 'USD' and char != 'SHOP':
      #     text = text.replace(char, 'name')
      if char in teencod.keys():
        text = text.replace(char, teencod[char])
      # char = char.replace('_', ' ')
      # char_lower = char.lower()
      # if char_lower in tenct:
      #   text = text.replace(char, 'name')
  text = text.replace('"', '')
  text = text.replace('”', '')
  text = text.replace('“', '')
  text = text.replace('.', ' ')
  text = text.replace(',', '')
  text = text.replace('(', '')
  text = text.replace(')', '')
  text = text.replace(':', '')
  text = text.replace('[', '')
  text = text.replace(']', '')
  text = text.replace('-', ' ')
  text = re.sub('\d{0,2}-?\d{0,2}\/\d{1,4}', 'date', text)
  for i in quy:
    text = text.replace(i, 'date')
  for i in year:
    text = text.replace(i, 'date')
  for i in month:
    text = text.replace(i, 'date')
  text = re.sub('\d+ năm ', 'date ', text)
  text = re.sub('\d+ tháng ', 'date ', text)
  text = re.sub(' \-?\d+\w?', ' number', text)
  text = text.split()
  for i in range(len(text)):
    if text[i].isdigit():
      text[i] = 'number'
  text = ' '.join(text)
  text1 = text.split()
  # for i in range(len(text1)+1):
  #   try:
  #     if text1[i][0].isupper() and text1[i+1][0].isupper():
  #       text = text.replace(text1[i], 'name')
  #       text = text.replace(text1[i+1], 'name')
  #   except:
  #     pass
  text = rdrsegmenter.tokenize(text)
  text = ' '.join([' '.join(x) for x in text])
  text = text.lower()
  return text

{'ctrai': 'con trai', 'khôg': 'không', 'bme': 'bố mẹ', 'cta': 'chúng ta', 'mih': 'mình', 'mqh': 'mối quan hệ', 'cgai': 'con gái', 'nhữg': 'những', 'mng': 'mọi người', 'svtn': 'sinh viên tình nguyện', 'r ': 'rồi', 'qtam': 'quan tâm', 'thươg': 'thương', 'qtâm': 'quan tâm', 'chug': 'chung', 'trườg': 'trường', 'thoy': 'thôi', 'đki': 'đăng ký', 'atsm': 'ảo tưởng sức mạnh', 'ạk': 'ạ', 'cv': 'công việc', 'vch': 'vãi chưởng', 'cùg': 'cùng', 'pn': 'bạn', 'pjt': 'biết', 'thjk': 'thích', 'keke': 'ce ce', 'ktra': 'kiểm tra', 'nek': 'nè', 'cgái': 'con gái', 'nthe': 'như thế', 'chúg': 'chúng', 'kái': 'cái', 'tìh': 'tình', 'phòg': 'phòng', 'lòg': 'lòng', 'từg': 'từng', 'rằg': 'rằng', 'sốg': 'sống', 'thuj': 'thôi', 'thuơng': 'thương', 'càg': 'càng', 'đky': 'đăng ký', 'bằg': 'bằng', 'sviên': 'sinh viên', 'ák': 'á', 'đág': 'đáng', 'nvay': 'như vậy', 'nhjeu': 'nhiều', 'xg': 'xuống', 'zồi': 'rồi', 'trag': 'trang', 'zữ': 'dữ', 'atrai': 'anh trai', 'kte': 'kinh tế', 'độg': 'động', 'lmht': 'liên minh huyền t

In [10]:
def predict(model, bpe, sense, vocab):
  subwords = '<s> ' + bpe.encode(sense) + ' </s>'
  encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
  encoded_sent = pad_sequences([encoded_sent], maxlen=195, dtype="long", value=0, truncating="post", padding="post")
  mask = [int(token_id > 0) for token_id in encoded_sent[0]]


  encoded_sent = torch.tensor(encoded_sent).cuda()
  mask = torch.tensor(mask).cuda()
  encoded_sent = torch.reshape(encoded_sent, (1, 195))
  mask = torch.reshape(mask, (1, 195))

  with torch.no_grad():
    outputs = model(encoded_sent, 
      token_type_ids=None, 
      attention_mask=mask)
    logits = outputs[0]
  return int(torch.argmax(logits))


In [11]:
sent = 'sản phẩm bị lỗi không cắm được tai nghe vào, shop giao không đúng màu'
predict(model, bpe, sent, vocab)

0

In [12]:
sent = 'sản phẩm tốt, giao hàng nhanh'
predict(model, bpe, sent, vocab)

1